In [1]:
from tensorflow import keras
import numpy as np

RND_SEED = 12345

def train_generator():
    return keras.preprocessing.image.ImageDataGenerator(
        validation_split=0.25,
        rescale=1./255.,
#        vertical_flip=True,
#        width_shift_range=0.2,
#        height_shift_range=0.2,
#		rotation_range=90,
    )

def load_train(path):
    return train_generator().flow_from_directory(
        path,
        target_size=(150, 150),
        batch_size=16,
        class_mode='sparse',
        subset='training',
        seed=RND_SEED)

def create_model(input_shape):
    model = keras.models.Sequential()
#    model.add(keras.layers.Input(input_shape))
    model.add(keras.layers.Conv2D(filters=6, kernel_size=(5, 5), padding='same', activation='tanh', input_shape=input_shape))
    model.add(keras.layers.AvgPool2D(pool_size=(2, 2)))
    model.add(keras.layers.Conv2D(filters=16, kernel_size=(5, 5), padding='valid', activation='tanh')) #, input_shape=(14, 14, 6)
    model.add(keras.layers.AvgPool2D(pool_size=(2, 2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=120, activation='tanh'))
    model.add(keras.layers.Dense(units=84, activation='tanh'))
    model.add(keras.layers.Dense(units=12, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy', metrics=['acc'])
    return model

def train_model(model, train_data, test_data, batch_size=None, epochs=5,
               steps_per_epoch=None, validation_steps=None):
    model.fit(train_data, 
              validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)
    return model

2025-09-16 05:49:30.364016: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-16 05:49:30.421425: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-16 05:49:31.767211: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
image_gen = train_generator()
train_datagen_flow = image_gen.flow_from_directory(
			'./datasets/fruits_small/',
			target_size=(150, 150),
			batch_size=16,
			class_mode='sparse',
			subset='training',
			seed=RND_SEED)
val_datagen_flow = image_gen.flow_from_directory(
			'./datasets/fruits_small/',
			target_size=(150, 150),
			batch_size=16,
			class_mode='sparse',
			subset='validation',
			seed=RND_SEED)

Found 1266 images belonging to 12 classes.
Found 417 images belonging to 12 classes.


In [3]:
features, target = next(train_datagen_flow)
len(train_datagen_flow)

80

In [4]:
model = create_model(features[0].shape)
model = train_model(model,
	train_data=train_datagen_flow,
	test_data=val_datagen_flow)

Epoch 1/5


/home/wtom/code/notebooks/practicum/vision/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-09-16 05:49:32.487625: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/wtom/code/notebooks/practicum/vision/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


80/80 - 11s - 134ms/step - acc: 0.1675 - loss: 2.4594 - val_acc: 0.0264 - val_loss: 2.6372
Epoch 2/5
80/80 - 12s - 146ms/step - acc: 0.2267 - loss: 2.2973 - val_acc: 0.2662 - val_loss: 2.1528
Epoch 3/5
80/80 - 10s - 128ms/step - acc: 0.3476 - loss: 1.9987 - val_acc: 0.3861 - val_loss: 1.8914
Epoch 4/5
80/80 - 12s - 151ms/step - acc: 0.6058 - loss: 1.2515 - val_acc: 0.7098 - val_loss: 0.9136
Epoch 5/5
80/80 - 13s - 168ms/step - acc: 0.7433 - loss: 0.7787 - val_acc: 0.7458 - val_loss: 0.7755
